## Q1. Running Elastic

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

In [1]:
!curl localhost:9200

{
  "name" : "838ec0ab7b66",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Swt3pPUwT0KvQz5KGkpwnQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


Answer: 42f05b9372a9a4a470db3b52817899b99a76ee73

## Getting the data

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
from elasticsearch8 import Elasticsearch
from tqdm.auto import tqdm

es_client = Elasticsearch('http://localhost:9200') 

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-homework01"

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

Q: Which function do you use for adding your data to elastic?

A: `index`

## Q3. Searching

In [8]:
query = "How do execute a command on a Kubernetes pod?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

es_client.search(index=index_name, body=search_query)

ObjectApiResponse({'took': 20, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions-homework01', '_id': 'gCuCV5cB0hTZ2kwq2Kco', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions-homework01', '_id': 'DyuCV5cB0hTZ2kwq2qg5', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving

Q: What's the score for the top ranking result?

A: `44.50556 (45.50)`

## Q4. Filtering

In [21]:
query = "How do copy a file to a Docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

search_results = es_client.search(index=index_name, body=search_query).body.get('hits', {}).get('hits', [])
search_results[2]

{'_index': 'course-questions-homework01',
 '_id': 'oCuCV5cB0hTZ2kwq2Kd-',
 '_score': 59.812744,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

Q: What's the 3rd question returned by the search engine?

A: "How do I copy files from a different folder into docker container’s working directory?"

## Q5. Building a prompt

In [36]:
context_template = ''

for entry in search_results:
    context_template += f"Q: {entry['_source']['question']}\n"
    context_template += f"A: {entry['_source']['text']}\n\n\n"

print(context_template.strip())

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani


Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how t

In [38]:
q5_question = "How do I execute a command in a running docker container?"

prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {q5_question}

CONTEXT:
{context_template}
""".strip()

# print(prompt_template)
len(prompt_template)

1464

Q: What's the length of the resulting prompt? (use the len function)

A: 1446 (1464)

## Q6. Tokens

In [40]:
import os
from anthropic import Anthropic
client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),  # This is the default and can be omitted
)

In [43]:
token_count = client.messages.count_tokens(
    model="claude-3-5-haiku-latest",
    messages=[
        {
            "role": "user",
            "content": prompt_template,
        }
    ],
)

In [44]:
token_count

MessageTokensCount(input_tokens=378)

Q: How many tokens does our prompt have?

A: Anthropic Haiku-3.5 returns `378` tokens. The closest answer in homework: `420`

## Bonus: generating the answer (ungraded)

In [46]:
response = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": prompt_template,
        }
    ],
    model="claude-3-5-haiku-latest",
)

print(response.content[0].text)

Based on the provided context, to execute a command in a running docker container, you can use the following steps:

1. First, find the container ID using:
   docker ps

2. Then execute the command using:
   docker exec -it <container-id> bash

This will launch an interactive bash shell in the running container, allowing you to run commands inside it.

Note: Replace <container-id> with the actual ID of the running container you want to interact with.


## Bonus: calculating the costs (ungraded)

In [50]:
# Haiku 3.5 price is $0.8 per 1M tokens
haiku_price = 0.8 / 10**6

print("Query cost is", haiku_price * 378, "USD")

Query cost is 0.00030240000000000003 USD
